In [ ]:
import os
import sys
from osgeo import ogr
os.environ['QT_QPA_PLATFORM'] = 'offscreen'
#sys.path.insert(0,'/usr/local/share/qgis/python')
#sys.path = ['C:/PROGRA~1/QGIS3~1.10/apps/qgis-ltr/./python', 'C:/Users/vitor/AppData/Roaming/QGIS/QGIS3\\profiles\\default/python', 'C:/Users/vitor/AppData/Roaming/QGIS/QGIS3\\profiles\\default/python/plugins', 'C:/PROGRA~1/QGIS3~1.10/apps/qgis-ltr/./python/plugins', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\Scripts', 'C:\\Program Files\\QGIS 3.10\\bin\\python37.zip', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\DLLs', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\lib', 'C:\\Program Files\\QGIS 3.10\\bin', 'C:\\Users\\vitor\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\lib\\site-packages', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\lib\\site-packages\\win32', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\lib\\site-packages\\win32\\lib', 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\lib\\site-packages\\Pythonwin', 'C:/Users/vitor/AppData/Roaming/QGIS/QGIS3\\profiles\\default/python', '.']
#os.environ['PATH'] = 'C:\\PROGRA~1\\QGIS3~1.10\\apps\\qgis-ltr\\bin;C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37;C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\Scripts;C:\\PROGRA~1\\QGIS3~1.10\\apps\\qt5\\bin;C:\\PROGRA~1\\QGIS3~1.10\\bin;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\system32\\WBem;C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\lib\\site-packages\\pywin32_system32;C:\\PROGRA~1\\QGIS3~1.10\\apps\\Python37\\lib\\site-packages\\numpy\\.libs'

from qgis.core import *
#QgsApplication.setPrefixPath("/usr/local", True) 
from qgis.gui import *
from qgis import processing
import qgis.utils
from qgis.PyQt.QtGui import QColor, QImage
from qgis.PyQt.QtCore import QSize, QBuffer, QIODevice, QVariant

#QgsApplication.setPrefixPath(r'C:/PROGRA~1/QGIS3~1.10\apps\qgis', True)
qgs = QgsApplication([], False)
qgs.initQgis()
print(QgsApplication.showSettings())

sys.path.append(r'C:/PROGRA~1/QGIS3~1.10\apps\qgis-ltr\python\plugins')


In [ ]:
covid_gpkg = r"C:\Users\vitor\Documents\MIEI\Tecnologias e Aplicações\cartografia\covid-pt-2020-04-20.gpkg"
conn = ogr.Open(covid_gpkg)
for i in conn:
    vlayer = QgsVectorLayer("{}|layername={}".format(covid_gpkg, i.GetName()), i.GetName(), "ogr")
    if not vlayer.isValid():
        print("Layer {} failed to load".format(i.GetName()))
    else:
        QgsProject.instance().addMapLayer(vlayer)
        print("Layer {} loaded".format(i.GetName()))
concelho = QgsProject.instance().mapLayersByName('concelho')[0]

In [ ]:
xt = concelho.extent()
# print(xt)
width  = 400
height = int(width*xt.height()/xt.width())
print("Gerar mapa com {} por {}".format(width, height))

options = QgsMapSettings()

options.setLayers([concelho])
options.setBackgroundColor(QColor(255, 255, 255))
options.setOutputSize(QSize(width, height))
options.setExtent(xt)
''''''        
render = QgsMapRendererParallelJob(options)
render.start()
render.waitForFinished()

image = render.renderedImage()


In [ ]:
from IPython.display import Image
imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image.save( imgbuf,"PNG" )
Image( imgbuf.data() )

In [ ]:
uri = "file:///C:/query.csv?encoding=%s&delimiter=%s&xField=%s&yField=%s&crs=%s" % ("UTF-8",",", "longitude", "latitude","epsg:4326")
print(uri)
#Make a vector layer
eq_layer=QgsVectorLayer(uri,"eq-data","delimitedtext")

#Check if layer is valid
if not eq_layer.isValid():
    print ("Layer not loaded")


In [ ]:
uri = r"file:///time_series_covid19_confirmed_global.csv?delimiter=,&xField=Long&yField=Lat"
print(uri)
confirmed = QgsVectorLayer(uri, "confirmed", "delimitedtext")
if not confirmed.isValid():
    print("Layer failed to load!")
uri = r"file:///time_series_covid19_deaths_global.csv?delimiter=,&xField=Long&yField=Lat"
print(uri)
deaths = QgsVectorLayer(uri, "deaths", "delimitedtext")
if not confirmed.isValid():
    print("Layer failed to load!")
uri = r"file:///time_series_covid19_recovered_global.csv?delimiter=,&xField=Long&yField=Lat"
print(uri)
recovered = QgsVectorLayer(uri, "recovered", "delimitedtext")
if not confirmed.isValid():
    print("Layer failed to load!")

In [ ]:
countries = r"C:/Users/vitor/Documents/MIEI/Tecnologias e Aplicações/cartografia/ne_10m_admin_0_countries.shp"
countries = QgsVectorLayer(countries, "countries", "ogr")
if not countries.isValid():
    print("Layer failed to load!")

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
df.head()

In [ ]:
xt = countries.extent()
# print(xt)
width  = 1200
height = int(width*xt.height()/xt.width())
print("Gerar mapa com {} por {}".format(width, height))

options = QgsMapSettings()

options.setLayers([res['OUTPUT']])
options.setBackgroundColor(QColor(255, 255, 255))
options.setOutputSize(QSize(width, height))
options.setExtent(xt)
''''''        
render = QgsMapRendererParallelJob(options)
render.start()
render.waitForFinished()

image = render.renderedImage()


In [ ]:
from IPython.display import Image
imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image.save( imgbuf,"PNG" )
Image( imgbuf.data() )

In [ ]:
from qgis.core import *
from qgis import *
 
Input_Table = r'C:/query.csv' # set the filepath for the input CSV
lon_field = 'longitude' # set the name for the field containing the longitude
lat_field = 'latitude' # set the name for the field containing the latitude
crs = 4326 # set the crs as needed
Output_Layer = 'output.shp' # set the filepath for the output shapefile
 
spatRef = QgsCoordinateReferenceSystem(crs, QgsCoordinateReferenceSystem.EpsgCrsId)
 
inp_tab = QgsVectorLayer(Input_Table, 'Input_Table', 'ogr')
prov = inp_tab.dataProvider()
fields = inp_tab.pendingFields()
outLayer = QgsVectorFileWriter(Output_Layer, None, fields, QGis.WKBPoint, spatRef)
 
pt = QgsPoint()
outFeature = QgsFeature()
 
for feat in inp_tab.getFeatures():
    attrs = feat.attributes()
    pt.setX(float(feat[lon_field]))
    pt.setY(float(feat[lat_field]))
    outFeature.setAttributes(attrs)
    outFeature.setGeometry(QgsGeometry.fromPoint(pt))
    outLayer.addFeature(outFeature)
del outLayer

In [ ]:
QgsProject().instance().addMapLayers([confirmed,countries])



In [ ]:
lien = QgsVectorLayerJoinInfo()
lien.setTargetFieldName('NAME')
lien.setJoinFieldName(r'Country/Region')
lien.setJoinLayerId(confirmed.id())
lien.setUsingMemoryCache(True)
lien.setJoinLayer(confirmed)
countries.addJoin(lien)

In [ ]:

for field in dale.fields():
    print(field.name(), field.typeName())

In [ ]:
graduated_renderer = QgsGraduatedSymbolRenderer()
# Add a few categories
graduated_renderer.addClassRange(QgsRendererRange(QgsClassificationRange('class 0-100', 0, 100), QgsMarkerSymbol()))
graduated_renderer.addClassRange(QgsRendererRange(QgsClassificationRange('class 101-200', 101, 200), QgsMarkerSymbol()))

for ran in graduated_renderer.ranges():
    print("{} - {}: {} {}".format(
        ran.lowerValue(),
        ran.upperValue(),
        ran.label(),
        ran.symbol()
      ))

In [ ]:


from qgis.PyQt import QtGui


myTargetField = r'confirmed_4/14/20'
myRangeList = []
myOpacity = 1
# Make our first symbol and range...
myMin = 0.0
myMax = 10000
myLabel = 'Group 1'
myColour = QtGui.QColor('#ffeeff')
mySymbol1 = QgsSymbol.defaultSymbol(countries.geometryType())
mySymbol1.setColor(myColour)
mySymbol1.setOpacity(myOpacity)
myRange1 = QgsRendererRange(myMin, myMax, mySymbol1, myLabel)
myRangeList.append(myRange1)
#now make another symbol and range...
myMin = 10000.1
myMax = 20000
myLabel = 'Group 2'
myColour = QtGui.QColor('#00eeff')
mySymbol2 = QgsSymbol.defaultSymbol(
     countries.geometryType())

mySymbol2.setColor(myColour)
mySymbol2.setOpacity(myOpacity)
myRange2 = QgsRendererRange(myMin, myMax, mySymbol2, myLabel)
myRangeList.append(myRange2)
myRenderer = QgsGraduatedSymbolRenderer('', myRangeList)
myClassificationMethod = QgsApplication.classificationMethodRegistry().method("EqualInterval")
myRenderer.setClassificationMethod(myClassificationMethod)
myRenderer.setClassAttribute(myTargetField)

countries.setRenderer(myRenderer)

In [ ]:
for ran in myRenderer.ranges():
    print("{} - {}: {} {}".format(
        ran.lowerValue(),
        ran.upperValue(),
        ran.label(),
        ran.symbol()
      ))

In [ ]:
for i in range(0,100):
    print(str(i)+": "+countries.attributeDisplayName(i))

In [ ]:
countries.startEditing()
countries.deleteAttributes(range(18,94))
countries.commitChanges()

In [ ]:
a=list()

In [ ]:
a

In [ ]:
tenta = QgsVectorLayer(countries)

In [ ]:
QgsAggregateCalculator.aggregate()

In [ ]:
countries.aggregate(aggregate=1,fieldOrExpression='LEVEL')

In [ ]:
help(QgsAggregateCalculator.Aggregate)

In [ ]:


from processing.core.Processing import Processing
Processing.initialize()
from processing.tools import *

layer1 = "path/to/point_shapefile.shp"
layer2 = "path/to/polygon_shapefile.shp"
result = "path/to/output_shapefile.shp"

general.runalg("qgis:joinattributesbylocation", countries, confirmed, u'intersects', 0, 0, '', 1, result)

In [ ]:
processing.alglist()

In [ ]:
import processing
from qgis.analysis import QgsNativeAlgorithms

from processing.core.Processing import Processing

Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
for alg in QgsApplication.processingRegistry().algorithms():
        print(alg.id(), "->", alg.displayName())

In [ ]:
processing.algorithmHelp("saga:rasterproduct")

In [ ]:
for alg in QgsApplication.processingRegistry().algorithms():
        print("{}:{} --> {}".format(alg, alg.name(), alg.displayName()))

In [ ]:
field = QgsField( 'PC', QVariant.String )

In [ ]:
confirmed.addExpressionField('concat(  "Province/State" , "Country/Region" )', field)
deaths.addExpressionField('concat(  "Province/State" , "Country/Region" )', field)
recovered.addExpressionField('concat(  "Province/State" , "Country/Region" )', field)

In [ ]:
help(QgsApplication.processingRegistry().addProvider)

In [ ]:
processing.algorithmHelp("qgis:joinattributestable")

In [ ]:
params = { 
    'INPUT' : confirmed,
    'FIELD' : 'PC',
    'INPUT_2' : deaths, 
    'FIELD_2' : 'PC',
    'FIELDS_TO_COPY' : '',
    'METHOD' : 0,
    'DISCARD_NONMATCHING' : False,
    'PREFIX' : 'deaths_',
    'NON_MATCHING' : 'memory:',
    'OUTPUT' : 'memory:'
}

In [ ]:
feedback = QgsProcessingFeedback()

res = processing.run("qgis:joinattributestable", params,feedback=feedback)


In [ ]:
params = { 
    'INPUT' : res['OUTPUT'],
    'FIELD' : 'PC',
    'INPUT_2' : recovered, 
    'FIELD_2' : 'PC',
    'FIELDS_TO_COPY' : '',
    'METHOD' : 0,
    'DISCARD_NONMATCHING' : False,
    'PREFIX' : 'recovered_',
    'NON_MATCHING' : 'memory:',
    'OUTPUT' : 'memory:'
}

In [ ]:
feedback = QgsProcessingFeedback()

res = processing.run("qgis:joinattributestable", params,feedback=feedback)


In [ ]:
dale = res['OUTPUT']

In [ ]:
processing.algorithmHelp("qgis:joinbynearest")

In [ ]:
params = { 
    'INPUT' : countries,
    'INPUT_2' : confirmed, 
    'FIELDS_TO_COPY' : '',
    'DISCARD_NONMATCHING' : False,
    'PREFIX' : 'confirmed_',
    'NEIGHBORS' : 1,
    'MAX_DISTANCE' : 99999999,
    'OUTPUT' : 'memory:'
}

In [ ]:
feedback = QgsProcessingFeedback()

res = processing.run('qgis:joinbynearest', params,feedback=feedback)

In [ ]:
vamos = res['OUTPUT']

In [ ]:
res


In [ ]:
vamos = res['OUTPUT']
camos = QgsVectorLayer(countries, "countries", "ogr")
if not countries.isValid():
    print("Layer failed to load!")

In [ ]:
processing.algorithmHelp("qgis:aggregate")

In [ ]:
params = { 
    'INPUT' : res['OUTPUT'],
    'GROUP_BY' : 'NAME', 
    'AGGREGATES':[
            {'aggregate': 'first_value', 'delimiter': ',', 'input': '"NAME"', 'length': 0, 'name': 'NAME', 'precision': 0, 'type': 10}, 
            {'aggregate': 'sum', 'delimiter': ',', 'input': '"confirmed_4/15/20"', 'length': 0, 'name': 'confirmed_4/15/20', 'precision': 0, 'type': 2},],
    'OUTPUT' : 'memory:'
}

In [ ]:
feedback = QgsProcessingFeedback()

res = processing.run('qgis:aggregate', params,feedback=feedback)

In [ ]:


from qgis.PyQt import QtGui


myTargetField = r'4/15/20'
myRangeList = []
myOpacity = 1
# Make our first symbol and range...
myMin = 0.0
myMax = 10000
myLabel = 'Group 1'
myColour = QtGui.QColor('#ffeeff')
mySymbol1 = QgsSymbol.defaultSymbol(res['OUTPUT'].geometryType())
mySymbol1.setColor(myColour)
mySymbol1.setOpacity(myOpacity)
myRange1 = QgsRendererRange(myMin, myMax, mySymbol1, myLabel)
myRangeList.append(myRange1)
#now make another symbol and range...
myMin = 10000.1
myMax = 20000
myLabel = 'Group 2'
myColour = QtGui.QColor('#00eeff')
mySymbol2 = QgsSymbol.defaultSymbol(
     res['OUTPUT'].geometryType())

mySymbol2.setColor(myColour)
mySymbol2.setOpacity(myOpacity)
myRange2 = QgsRendererRange(myMin, myMax, mySymbol2, myLabel)
myRangeList.append(myRange2)
myRenderer = QgsGraduatedSymbolRenderer('', myRangeList)
myClassificationMethod = QgsApplication.classificationMethodRegistry().method("EqualInterval")
myRenderer.setClassificationMethod(myClassificationMethod)
myRenderer.setClassAttribute(myTargetField)

res['OUTPUT'].setRenderer(myRenderer)